# *Модель*

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        """
        Архитектура:
        - Слой 1: Сверточный слой (conv1) с 8 фильтрами 3x3, шаг 1, padding 1.
        - Слой 2: MaxPooling с ядром 2x2 и шагом 2.
        - Слой 3: Сверточный слой (conv2) с 16 фильтрами 3x3, шаг 1, padding 1.
        - Слой 4: MaxPooling с ядром 2x2 и шагом 2.
        - Полносвязный слой: Выход 10 классов.
        """
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        """
        Forward-проход: обработка входных данных через слои.
        """
        x = F.relu(self.conv1(x))  # Первый сверточный слой + активация ReLU
        x = self.pool(x)           # MaxPooling
        x = F.relu(self.conv2(x))  # Второй сверточный слой + активация ReLU
        x = self.pool(x)           # MaxPooling
        x = x.view(x.size(0), -1)  # Преобразование вектора в одномерный (flatten)
        x = self.fc1(x)            # Полносвязный слой
        return x

*Определяем архитекутуру модели.*

*Что за ```nn.Module``` оно наследуется из pytorch и представляет собой набор инструментов для создания и управления архитектурой своей нейронной сети*

*Начну с самого начала, ```in_channels=1``` - значит что у нас будут только черно белые изображения, подобное выбранное значение определяет то что у нас будет только один цвет и разве что мы будем менять только ее яркость. Если совсем коротко, то наличие одного канала, говорит нам о том что мы можем управлять только интенсивностью черного цвета*

*```num_classes=10``` - является чем то вроде указателен на то сколько классов модель должна различать, в данном случае можно прямо сказать что от 0 до 9, модель должна четко отличать между собой и определять*
*Приметивны пример отбора наиболее вероятного варианта моделью `[0.1, 0.05, 0.05, 0.8, 0.01, 0.02, 0.02, 0.02, 0.01, 0.01]` под индексом 3 наибольшая реакция*

*```super(CNN, self).__init__()``` инициализиируем вызов функций которые определяли для этого класса ранее, для внедрения нашей последующей конфигруации работы алгоритма и в целом его взаимодействие с аппаратной частью*




*`nn.Conv2d` - обозночает сверточный слой для работы с двумерными данными*
каналы уже разбирали, их 1 штук, дальше `8` - является значением числом ядер или же число значений которое будет излвекаться на выявление признаков.
`kernel_size=3` - 3 обозначает 3x3, это диапозон охвата одного ядра который обучается на текущем слое, который сканирует изображение.
`stride=1` - шаг ядра, то, с какое расстояние будет преодолевать ядра в пикселях за один ход, в нашем примере, это означает что ядра будут двигаться на 1 пиксель за каждый ход.
`padding=1` - добавляет пустых пикселей вокруг изображения, что бы сохранить ее целостность и не налазить никуда

> ну по своей сути, получается, что 8 так называемых ядер но по своей сути выступают сканерами, которые имеют размерность 3х3 а благодаря padding реальная их площадь 4х4, поскольку есть невидемое утолщение для сохранения целостности, по stride они ходят за один ход на один пиксель. Оно так же ведет поиски в диапозоне своих 3х3, лишь с учетом дополнительных страхоночных пикселей, оно не составляет 4х4 фактический



`nn.MaxPool2d` - если браться за тафталогию, то это является областью максимального пулинга, в котором если говорить грубо то каждое ядро сжымается и в дальнейшем из них с каждого диапозона которые охватили путем прохождения по определенным длинам шагов и массой ядра будут взяты максимальные числа как представители их зоны

*и так `conv2`, здесь, входное количество каналов ровно пропорциально выходному количеству каналов их предыдущего сверточника. Что касатель `16` то это количество ядер или возможно будет менее корректно но легче `сканнеров`, остальное все базовое, диапозон сканнера 3х3, шаг в 1 и защитный слой 1.*

*`Linear` - ялвяется полносвязным слоем, который выполняет линейное переобразование выходных данных при помощи формулы. На который и передаюутся все значения вместе с количеством классов модели. Является конечной точкой, в котором оно собирает все значения после пулингов и решает, каким числом вероятнее всего является.*

*функция `forward`, является чем то вроде маршрутк, который проводит x - в лице нашей картинки, через все этапы которые мы загатавлевали ранее. Отдельно стоит отметить `Flatten`, поскольку видем впервые, это является подведением итогов, где все признаки суммируются и преобразуются в финальный ответ*

In [ ]:
def train_model():
    # Устройство для вычислений (CPU/GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Гиперпараметры
    learning_rate = 0.001
    batch_size = 64
    num_epochs = 10

    # Загрузка данных MNIST
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transform, download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    # Инициализация модели, функции потерь и оптимизатора
    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Цикл обучения
    for epoch in range(num_epochs):
        print(f"Эпоха [{epoch + 1}/{num_epochs}]")
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            data, targets = data.to(device), targets.to(device)

            # Forward-проход
            scores = model(data)
            loss = criterion(scores, targets)

            # Обратное распространение и обновление весов
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Сохранение обученной модели
    torch.save(model.state_dict(), "mnist_cnn.pth")
    print("Модель сохранена как 'mnist_cnn.pth'")
    return model

*обучение и тренировка модели*

*на первой страке можем заметить, что программа определяет наличие `cuda` ядер, в простонародии видеокарту, при отсутсвтии, использует возможности процессора, хочу отметить что выглядит достаточно удобно, так как подстраивается само*

*что касается гиперпараметров, `learning_rate` - отвечает за скорость обучения, где мы выбрали достаточное малое значение, что бы обучение модель происходило достаточно размеренно и точно, для скорости можно поднять, но потери будут все выше. batch_size - явдяется чем то вроде мультипоточности, количество того сколько алгоритм обрабатывает за одну итерацию можно сказать. num_epochs - определеяет количество того, сколько раз алгоритм пройдет тренировку, значение 10 может показаться большим, но кажется самый среднячок*

*загрузка тренировочного датасета от MNIST, это является стандартным набором данных для подобных задач, с числами размерностью 28на28*

*следом идем этап инициализации и применение нашей архитектуры, первой стрчокой сразу видем, как в переменную model инициализируем класс `CNN`, с переносом посредством `.to()`, на нужный нам исполнтель в лице процессора либо видекарты в зависимотси от выполненйи условий. Следом идет функция потерь, которая сравнивает предсказания модели с заведомо верными метками используя для задач классификации. Дальше `.Adam`, который автоматическим образом, обновляет веса модели на основе ошибки, Adam автоматически регулирует шаг обновления для каждого веса, посредством переданного ему parameters*

*следом идет цикл обучения, который запускается по количеству эпох, этох это полный проход по всему алгоритму обчения*

*ну и, сохранение модели в файле (mnist_cnn.pth)*



In [ ]:
def predict_image(image):
    """
    Принимает изображение (PIL Image), обрабатывает его и возвращает предсказанный класс.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = CNN()
    model.load_state_dict(torch.load("mnist_cnn.pth", map_location=device))
    model.to(device)
    model.eval()

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        scores = model(image)
        _, prediction = scores.max(1)
    return prediction.item()

*итак, функция предсказания*

*начать стоит с того, что нас встречает определение вычеслительной аппаратуры, в лице видеоядра тобе `CUDA` или же `CPU` процессор, стоит отметить, что очень удобно то что библиотека предоставляет возмность не отходя от кассы провети проврку.*

*иницализируем архитекутуру модели в переменную `model`, но пока что она находится без самой обученной модели, мы лишь определили ее архитектуру.*

*следом вытиягиваем файл с моделью `mnist_cnn.pth`, которую обучали на предыдущем этапе*

*ну и конечно же выбор того на каком аппаратном ускорителе будет работать при помощи `.to(и переменная в которую мы ранее определяли проверку)`*

*`.eval()`, является чем то вроде переключателя режима нейронной сети, модели могут работать по разному в зависимости от того мы их обучаем или используем на практике. Если приводить в пример то когда она может пригодиться, то тот же `dropout`, который при обучении модели при достижении определенного придела может обнулить значения нейронов воизбежание переобучения, тут оно нам будет только мешать, при `eval`, мы будем использовать все нейроны которые имеем*

*следом, у нас идет предварительная адаптация изображения в лице transform, является чем то вроде процессинга из текста, который оптимизирует и подстраивает под возможности модели*

*`transforms.Grayscale(num_output_channels=1),`, преобразует изображение в однокальный формат, что бы изображение было в пределах белого и черного*

*`transforms.Resize((28, 28)),`, полученное изображение масштабируется до разрешения 28на28, с этого следует вывод, что модель обучалась на подобных изображениях*

*`transforms.ToTensor()`, переобразует в тензор pytorch, что бы работать с числовыми данными подсчитывать вероятности*

*`transforms.Normalize((0.5,), (0.5,))`, приводит значения пикселей к одному диапозону*

*`transform(image)` - примененние нормализации на изображении*

*`unsqueeze(0)` - необходимо, для того что бы модель приняла изображение отправилось в формате одного батча*

*`.to(device)` - определение аппаратного ускорителя*

*`with torch.no_grad():` - отключение автоматического вычисления градиента. Для чего это нужно? обычно требуется при обучении, но в нашем случае, мы выносим предикт на модели*

*`scores = model(image)` - отправка изображения в модель*

*`        _, prediction = scores.max(1)`*
*    `return prediction.item()` - определяет наиболее вероятное значение из 10 эпох и получается суммарно 100 индексов, поскольку было 10 классов по 10 эпох*

In [ ]:
def check_accuracy(loader, model):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()  # Переключение модели в режим оценки

    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    accuracy = float(num_correct) / float(num_samples) * 100
    print(f"Точность: {accuracy:.2f}%")
    model.train()  # Возвращение в режим обучения

*Оценка точности модели*

*первые две строки уже знакомы нам, `определение аппаратного ускорителя` для обработки и переключение модели в `режим оценки`.*

*`num_correct = 0` - счетчик верных предсказаний модели*

*`num_samples = 0` - общее количество проверенных примеров*

*данные параметры будут использоваться для рассчета точности*


*`with torch.no_grad():` - как говорил ранее, отключение автоматического определителя градиентов, позволяет разгрузить память и обеспечивает более быструю работу распознавания*


*`for x, y in loader:` - проходит через весь набор данных, `x` - является входным изображением которое мы ранее переопределили в батч, следом идет `y` - в которой метки классов от MNIST, которые мы обучали*

*`x, y = x.to(device), y.to(device)` - переносит входные данные и классы (метки) на то же устройство на котором работает модель*

*`scores = model(x)` - пропускает входные батчи через модель. Возвращается данные в следующем виде `[1.5, 2.3, 0.7, 4.0, 0.5, 0.1, 0.2, 0.8, 1.0, 0.3]`, где самая максимальная оценка означает самую вероятную*

*`_, predictions = scores.max(1)` - возвращает максимальное значение вместе с его индексом, почему не использовали просто max()? в таком случае он бы вернул только само максимальное значение*

*`num_correct += (predictions == y).sum()` - сравнивает предсказания модели с классами из MNIST, которые хранились в `y`. Оно накапливает в себе булевые значения*

*`num_samples += predictions.size(0)` - возвращает количество примеров в батче*

*`accuracy = float(num_correct) / float(num_samples) * 100` - рассчет точности*

*`model.train()` - возвращение модели в режим обучения*

In [ ]:
if __name__ == "__main__":
    # Обучение модели
    model = train_model()

    # Тестирование модели
    transform = transforms.Compose([transforms.ToTensor()])
    test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transform, download=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

    check_accuracy(test_loader, model)

*запускает процесс сборки модли, если была запущенна как основная программа*

*`model = train_model()` - ининциализируется модель в тестовом формате*

*Стягиваются датасеты и последним подсчитывается точность модели*

# *Интерфейс*

In [ ]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
from model import predict_image  # Импорт функции предсказания из model.py

*инструменты необходимые для воспроизведение интейрфейса и работы с изображениям, а так же функция выношения предикта из скрипта модели*

In [ ]:
class DrawApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Digit Recognizer")

        # Canvas для рисования
        self.canvas = tk.Canvas(root, width=280, height=280, bg="white")
        self.canvas.pack()

        # Кнопки
        self.button_predict = tk.Button(root, text="Recognize", command=self.recognize)
        self.button_predict.pack()

        self.button_clear = tk.Button(root, text="Clear", command=self.clear_canvas)
        self.button_clear.pack()

        # Метка для отображения результата
        self.label_result = tk.Label(root, text="Draw a digit and press 'Recognize'")
        self.label_result.pack()

        # Изображение и инструмент рисования
        self.image = Image.new("RGB", (280, 280), "white")
        self.draw = ImageDraw.Draw(self.image)

        # Привязка событий рисования
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        x, y = event.x, event.y
        r = 6  # Размер кисти
        self.canvas.create_oval(x - r, y - r, x + r, y + r, fill="black", outline="black")
        self.draw.ellipse([x - r, y - r, x + r, y + r], fill="black")

    def recognize(self):
        # Преобразование изображения в градации серого
        gray_image = ImageOps.grayscale(self.image)
        result = predict_image(gray_image)  # Вызов функции из model.py
        self.label_result.config(text=f"Recognized digit: {result}")

    def clear_canvas(self):
        # Очистка холста и изображения
        self.canvas.delete("all")
        self.image = Image.new("RGB", (280, 280), "white")
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(text="Draw a digit and press 'Recognize'")

*основные настройки мини прилоежния и рисования на ней, вместе с функцией стерки*

*их интересного стоит отметить функцию `recognize`*
которая как раз таки и применяет функцию для выношения предиктов, предварительно переработав в черно белый формат, при помощи `.grayscale(self.image)`*

*следом идет отображение метки при помощи `label_result.config`*

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawApp(root)
    root.mainloop()

*инициализация всего скрипта*